<a href="https://colab.research.google.com/github/IsaacFigNewton/DisCoFuzz/blob/main/DisCoFuzz_WF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and Config

## Download, install dependencies

In [1]:
!pip install POT

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.5 MB/s eta 0:00:00


## Import

In [ ]:
from typing import Callable, Dict, List, Tuple, Set
import math
import numpy as np
import random
import nltk

nltk.download('wordnet')
nltk.download('framenet-17')
nltk.download('stopwords')
from nltk.corpus import wordnet as wn
from nltk.corpus import framenet as fn
from nltk.corpus import stopwords

import spacy

import pandas as pd

import scipy.stats as stats
from scipy.special import expit, logit

import matplotlib.pyplot as plt

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA

import tensorflow as tf

import ot

from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Error loading framenet-17: Package 'framenet-17' not found
[nltk_data]     in index
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Config

In [ ]:
stop_words = stopwords.words("english")
all_synsets = list(wn.all_synsets())

In [ ]:
wn_dict = {
    'synset': list(),
    'lemmas': list(),
    'antonyms': list(),
    'gloss': list(),
    "cleaned_gloss": list(),
}

for s in all_synsets:
  synset_lemma = s.name().split(".")[0]
  lemmas = s.lemmas()
  antonyms = list()
  for lemma in lemmas:
    antonyms += list(lemma.antonyms())
  if len(lemmas) > 1 and len(antonyms) > 0:
    wn_dict['synset'].append(s.name())
    wn_dict['lemmas'].append([l.name() for l in lemmas if l.name() != synset_lemma])
    wn_dict['antonyms'].append([a.name() for a in antonyms])
    wn_dict['gloss'].append(s.definition())
    cleaned_gloss = " ".join([t for t in s.definition().split(" ") if t not in stop_words])
    wn_dict['cleaned_gloss'].append(cleaned_gloss.replace("_", " "))

wn_df = pd.DataFrame.from_dict(wn_dict)
wn_df.head()

In [ ]:
wn_df = wn_df.sample(frac=0.05).reset_index(drop=True)
sample_size = len(wn_df)
print(f'Number of synsets: {len(wn_df)}')

In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPU available: {gpus}")

# Classes

## LemmaVectorizer

In [ ]:
class LemmaVectorizer:
    def __init__(self, keyed_vectors:dict=None):
        self.embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
        self.keyed_vectors = keyed_vectors if keyed_vectors else dict()

    def __call__(self, X: str) -> np.ndarray:
        v = self.keyed_vectors.get(X)#.lemma_.lower())
        if v is not None:
            return np.asarray(v, dtype=float)

        # if X not in self.keyed_vectors, add it
        self.keyed_vectors[X] = self.embedding_model.encode(X)
        return self.keyed_vectors[X]

## FourierPDF
for probability density function operations

In [ ]:
class FourierPDF:
  def __init__(self, kernel_size:int):
    self.kernel_size = kernel_size
    # get frequencies
    self.k_values = tf.cast(
        tf.range(0, kernel_size),
        tf.complex64
    )
    # pre-computer partial divisor for faster integration
    #   shape=(, self.kernel_size)
    # add 1e-20 to avoid division by 0
    self.divisor = tf.expand_dims(
        self.k_values+1e-20,
        axis=0
    )

  def _get_cdf_batch(self, a: tf.Tensor) -> tf.Tensor:
      """
      Batch integration.
      a: shape (batch_size, kernel_size)
      Returns: shape (batch_size,)
      """
      if len(tf.shape(a)) != 2:
          raise ValueError(f"Input tensor must have shape (batch_size, kernel_size), received tensor of shape {tf.shape(a)}")
      if a.dtype != tf.complex64:
          raise ValueError(f"Input tensors must be complex64, received {a.dtype}")

      batch_size = tf.shape(a)[0]
      # Create indices for the k=0 term for each item in the batch
      indices = tf.stack([tf.range(batch_size), tf.zeros(batch_size, dtype=tf.int32)], axis=1)

      # get AC term (just set all k=0 values to 0)
      AC = tf.tensor_scatter_nd_update(
          a,
          indices,
          tf.zeros(batch_size, dtype=tf.complex64)
      )
      # get divisors
      divisor = tf.broadcast_to(
          self.divisor,
          [tf.shape(a)[0], self.kernel_size]
      )
      AC = AC / divisor

      # add the k=0 terms back in
      return tf.tensor_scatter_nd_update(
          AC,
          indices,
          a[:, 0]
      )

  def _integrate_batch(self, a: tf.Tensor, ub:float=1, lb:float=0) -> tf.Tensor:
      """
      Batch integration.
      a: shape (batch_size, kernel_size)
      Returns: shape (batch_size,)
      """
      if len(tf.shape(a)) != 2:
          raise ValueError(f"Input tensor must have shape (batch_size, kernel_size), received tensor of shape {tf.shape(a)}")
      if a.dtype != tf.complex64:
          raise ValueError(f"Input tensors must be complex64, received {a.dtype}")

      batch_size = tf.shape(a)[0]

      # Create indices for the k=0 term for each item in the batch
      indices = tf.stack([tf.range(batch_size), tf.zeros(batch_size, dtype=tf.int32)], axis=1)

      # get AC term (just set all k=0 values to 0)
      AC = tf.tensor_scatter_nd_update(
          a,
          indices,
          tf.zeros(batch_size, dtype=tf.complex64)
      )
      # get divisors
      divisor = tf.broadcast_to(
          self.divisor,
          [tf.shape(a)[0], self.kernel_size],
      )

      upper_bound = tf.exp(divisor * ub)
      lower_bound = tf.exp(divisor * lb)
      diff = upper_bound - lower_bound

      integrals_k_nonzero = tf.reduce_sum(
          (AC / divisor) * diff,
          axis=1
      )

      # Combine k=0 and k>0 terms
      return a[:, 0] + integrals_k_nonzero

  def _integrate(self, a: tf.Tensor, ub:float=1, lb:float=0) -> tf.Tensor:
      """
      Single integration helper.
      a: shape (kernel_size)
      Returns: scalar
      """
      if len(tf.shape(a)) != 1:
          raise ValueError(f"Input tensor must have shape (kernel_size,), received tensor of shape {tf.shape(a)}")
      if a.dtype != tf.complex64:
          raise ValueError(f"Input tensors must be complex64, received {a.dtype}")

      a_batch = tf.expand_dims(a, axis=0)
      result_batch = self._integrate_batch(a_batch, ub, lb)
      return tf.squeeze(result_batch)

  def _normalize_batch(self, a: tf.Tensor) -> tf.Tensor:
      """
      Batch normalization of probability density functions.
      a: shape (batch_size, kernel_size)
      """
      if len(tf.shape(a)) != 2:
          raise ValueError(f"Input tensor must have shape (batch_size, kernel_size), received tensor of shape {tf.shape(a)}")
      if a.dtype != tf.complex64:
          raise ValueError(f"Input tensor must be complex64, received {a.dtype}")

      total_integral = self._integrate_batch(a)  # (batch_size,)
      total_integral = tf.expand_dims(total_integral, 1)  # (batch_size, 1)

      # Avoid division by zero
      total_integral = tf.where(
          tf.abs(total_integral) > 1e-10,
          total_integral,
          tf.ones_like(total_integral)
      )

      return a / total_integral

  def _normalize(self, a: tf.Tensor) -> tf.Tensor:
      """
      Single normalization helper for normalizing probability density functions.
      a: shape (kernel_size,)
      Returns: shape (kernel_size,)
      """
      if len(tf.shape(a)) != 1:
          raise ValueError(f"Input tensor must have shape (kernel_size,), received tensor of shape {tf.shape(a)}")
      if a.dtype != tf.complex64:
          raise ValueError(f"Input tensor must be complex64, received {a.dtype}")

      a_batch = tf.expand_dims(a, axis=0)
      result_batch = self._normalize_batch(a_batch)
      return tf.squeeze(result_batch, axis=0)

  def _convolve_batch(self, a: tf.Tensor, b: tf.Tensor) -> tf.Tensor:
      """
      Batch convolution using FFT.
      a, b: shape (batch_size, kernel_size)
      Returns: shape (batch_size, kernel_size)
      """
      if len(tf.shape(a)) != 2:
          raise ValueError(f"Input tensor must have shape (batch_size, kernel_size), received tensor of shape {tf.shape(a)}")
      if len(tf.shape(b)) != 2:
          raise ValueError(f"Input tensor must have shape (batch_size, kernel_size), received tensor of shape {tf.shape(b)}")

      # Batch FFT convolution
      A_fft = tf.signal.fft(tf.cast(a, tf.complex64))
      B_fft = tf.signal.fft(tf.cast(b, tf.complex64))
      C_fft = A_fft * B_fft
      C = tf.signal.ifft(C_fft)

      return tf.cast(C, tf.complex64)

  def _convolve(self, a: tf.Tensor, b: tf.Tensor) -> tf.Tensor:
      """
      Single convolution helper.
      a, b: shape (kernel_size,)
      Returns: shape (kernel_size,)
      """
      a_batch = tf.expand_dims(a, axis=0)
      b_batch = tf.expand_dims(b, axis=0)
      result_batch = self._convolve_batch(a_batch, b_batch)
      return tf.squeeze(result_batch, axis=0)

  def _differentiate_batch(self, a: tf.Tensor) -> tf.Tensor:
      """
      Batch differentiation.
      a: shape (batch_size, kernel_size)
      Returns: shape (batch_size, kernel_size)
      """
      if len(tf.shape(a)) != 2:
          raise ValueError(f"Input tensor must have shape (batch_size, kernel_size), received tensor of shape {tf.shape(a)}")

      return a * tf.exp(1j * self.k_values)

  def _differentiate(self, a: tf.Tensor) -> tf.Tensor:
      """
      Single differentiation helper.
      a: shape (kernel_size,)
      Returns: shape (kernel_size,)
      """
      a_batch = tf.expand_dims(a, axis=0)
      result_batch = self._differentiate_batch(a_batch)
      return tf.squeeze(result_batch, axis=0)

## Fuzzification Classes

### FuzzyFourierSetMixin

In [ ]:
class FuzzyFourierSetMixin(FourierPDF):
  def __init__(self, sigma: float, kernel_size: int):
      if kernel_size < 1:
          raise ValueError("Kernel size must be at least 1")
      super().__init__(kernel_size)
      self.sigma = tf.constant(sigma, dtype=tf.complex64)

      # Pre-compute Fourier coefficients for all k values
      # C_k = e^{-\frac{a^{2}k^{2}}{2}} and keep mu portion separate for now
      c_k = tf.exp(-0.5 * (self.sigma ** 2) * (self.k_values ** 2))
      self.fourier_coeffs = c_k / (2 * np.pi)

  def _get_gaussian_at_mu_batch(self, mu: tf.Tensor) -> tf.Tensor:
      """
      Batch computation of Fourier series for Gaussians centered at multiple mu values.
      mu: shape (batch_size,)
      Returns: shape (batch_size, kernel_size)
      """
      # mu part of C_n = e^{-ikb}
      #   combine mu.shape = (batch_size,) with self.k_values.T.shape = (,self.kernel_size)
      #   to get tensor of shape (batch_size, self.kernel_size)
      mu_c_k = tf.linalg.matmul(
          tf.expand_dims(tf.cast(mu, dtype=tf.complex64), axis=1),
          tf.expand_dims(self.k_values, axis=0)
      )
      mu_c_k = tf.exp(-1j * mu_c_k)

      # combine (batch_size, self.kernel_size) with (self.kernel_size,) self.fourier_coeffs
      return self._normalize_batch(mu_c_k * self.fourier_coeffs)

  def fuzzify(self, component: float) -> tf.Tensor:
      """
      Convert a real-valued component to a Fourier series representation of a Gaussian.
      component: scalar float
      Returns: shape (kernel_size,)
      """
      mu = tf.constant([component], dtype=tf.float32)
      return self._fuzzify_batch(mu)

  def fuzzify_batch(self, components: tf.Tensor) -> tf.Tensor:
      """
      Batch fuzzification.
      components: shape (batch_size,)
      Returns: shape (batch_size, kernel_size)
      """
      if len(tf.shape(components)) != 1:
          raise ValueError(f"Input tensor must have shape (batch_size,), received tensor of shape {tf.shape(components)}")

      return self._get_gaussian_at_mu_batch(components)

  def negation(self, a: tf.Tensor) -> tf.Tensor:
      """
      Fuzzy negation: NOT(a) = 1 - a
      a: shape (kernel_size,)
      Returns: shape (kernel_size,)
      """
      # Create representation of constant function 1
      one = tf.zeros_like(a)
      one = tf.tensor_scatter_nd_update(
          one,
          [[0]],
          [tf.constant(1.0, dtype=tf.complex64)]
      )

      return self._normalize(one - a)

  def negation_batch(self, a: tf.Tensor) -> tf.Tensor:
      """
      Batch fuzzy negation: NOT(a) = 1 - a
      a: shape (batch_size, kernel_size)
      Returns: shape (batch_size, kernel_size)
      """
      if len(tf.shape(a)) != 2:
          raise ValueError(f"Input tensor must have shape (batch_size, kernel_size), received tensor of shape {tf.shape(a)}")
      batch_size = tf.shape(a)[0]

      # Create batch of constant function 1
      # base tensor of same shape as a
      ones = tf.zeros_like(a)
      # indices at which to place 1's
      indices = tf.stack([
          tf.range(batch_size),
          tf.zeros(batch_size, dtype=tf.int32)
      ], axis=1)
      # ones to be inserted at said indices
      updates = tf.ones([batch_size], dtype=tf.complex64)
      # update the base tensor with 1's
      ones = tf.tensor_scatter_nd_update(ones, indices, updates)

      # get normalized negation
      return self._normalize_batch(ones - a)

  def intersection(self, a: tf.Tensor, b: tf.Tensor, normalize: bool = True) -> tf.Tensor:
      """
      Fuzzy intersection using product (min approximation).
      a, b: shape (kernel_size,)
      Returns: shape (kernel_size,)
      """
      result = self._convolve(a, b)
      if normalize:
          result = self._normalize(result)
      return result

  def intersection_batch(self, a: tf.Tensor, b: tf.Tensor, normalize: bool = True) -> tf.Tensor:
      """
      Batch fuzzy intersection using product.
      a, b: shape (batch_size, kernel_size)
      Returns: shape (batch_size, kernel_size)
      """
      if len(tf.shape(a)) != 2:
          raise ValueError(f"Input tensor must have shape (batch_size, kernel_size), received tensor of shape {tf.shape(a)}")
      result = self._convolve_batch(a, b)
      if normalize:
          result = self._normalize_batch(result)
      return result

  def union(self, a: tf.Tensor, b: tf.Tensor, normalize: bool = True) -> tf.Tensor:
      """
      Fuzzy union: a + b - a*b
      a, b: shape (kernel_size,)
      Returns: shape (kernel_size,)
      """
      convolved = self._convolve(a, b)
      result = a + b - convolved
      if normalize:
          result = self._normalize(result)
      return result

  def union_batch(self, a: tf.Tensor, b: tf.Tensor, normalize: bool = True) -> tf.Tensor:
      """
      Batch fuzzy union: a + b - a*b
      a, b: shape (batch_size, kernel_size)
      Returns: shape (batch_size, kernel_size)
      """
      if len(tf.shape(a)) != 2:
          raise ValueError(f"Input tensor must have shape (batch_size, kernel_size), received tensor of shape {tf.shape(a)}")
      convolved = self._convolve_batch(a, b)
      result = a + b - convolved
      if normalize:
          result = self._normalize_batch(result)
      return result

### FourierFuzzifier

WF geodesic calculation functions based on [the work by Elsa Cazelles, Arnaud Robert, Felipe Tobar](https://github.com/GAMES-UChile/Wasserstein-Fourier.git) in their paper ["The Wasserstein-Fourier Distance for Stationary Time Series"](https://arxiv.org/abs/1912.05509) to make it compatible with the component-wise series representation I'm using for gaussians.

In [ ]:
import tensorflow as tf
import numpy as np
from typing import List, Union
import ot

class FourierFuzzifier(FuzzyFourierSetMixin):
    """TensorFlow-accelerated version of FourierFuzzifier with set operations"""

    def __init__(self, sigma: float, kernel_size: int):
        if kernel_size < 1:
            raise ValueError("Kernel size must be at least 1")
        super().__init__(sigma, kernel_size)

    def get_npsd_batch(self, a: tf.Tensor) -> tf.Tensor:
        # normalize the power spectral densities
        if len(tf.shape(a)) != 2:
            raise ValueError(f"Input tensor must have shape (batch_size, kernel_size), received tensor of shape {tf.shape(a)}")
        norms = tf.expand_dims(tf.math.reduce_sum(tf.abs(a), axis=1), axis=1)
        norms = tf.broadcast_to(
            norms,
            [tf.shape(a)[0], tf.shape(a)[1]]
        )
        return tf.abs(a) / norms

    def similarity(self, a: tf.Tensor, b: tf.Tensor, method: str) -> Union[float, np.ndarray]:
        """
        Compute similarity as ot similarity in frequency domain.
        a, b: shape (kernel_size,)
        Returns: scalar similarity
        """
        if not a or not b:
          return None

        if len(tf.shape(a)) != 2:
            raise ValueError(f"Input tensor must have shape (batch_size, kernel_size), received tensor of shape {tf.shape(a)}")
        if len(tf.shape(b)) != 2:
            raise ValueError(f"Input tensor must have shape (batch_size, kernel_size), received tensor of shape {tf.shape(b)}")

        return self.similarity_batch(
            tf.expand_dims(a, axis=0),
            tf.expand_dims(b, axis=0),
            method=method
        )

    def similarity_batch(self, a: tf.Tensor, b: tf.Tensor, method: str) -> Union[float, np.ndarray]:
        """
        Batch computation of pairwise similarities.
        a, b: shape (batch_size, kernel_size)
        Returns: shape (batch_size,) - similarity for each pair
        """
        if len(tf.shape(a)) != 2:
            raise ValueError(f"Input tensor must have shape (batch_size, kernel_size), received tensor of shape {tf.shape(a)}")
        if len(tf.shape(b)) != 2:
            raise ValueError(f"Input tensor must have shape (batch_size, kernel_size), received tensor of shape {tf.shape(b)}")

        flatten_layer = tf.keras.layers.Flatten()
        freqs = tf.range(0, self.kernel_size, dtype=tf.float32)

        if method == "npsd-ot":
            # get normalized power density spectra
            a_npsd = self.get_npsd_batch(a)
            b_npsd = self.get_npsd_batch(b)

            # # energy is proportional to frequency^2 * amplitude^2
            # a_energy = a_npsd * a_npsd * freqs * freqs
            # b_energy = b_npsd * b_npsd * freqs * freqs
            # # Normalize to probability distributions
            # a_energy = a_energy / tf.reduce_sum(a_energy, axis=1, keepdims=True)
            # b_energy = b_energy / tf.reduce_sum(b_energy, axis=1, keepdims=True)

            max_freq = self.kernel_size - 1
            a_npsd = a_npsd.numpy()
            b_npsd = b_npsd.numpy()

            u, v = np.meshgrid(freqs, freqs)
            # normalize Wasserstein-2 metric
            cost = np.exp(u - v)
            # cost = (u - v)**2
            cost = np.ascontiguousarray(cost, dtype='float64')

            total_cost = 0
            for i in range(a.shape[0]):
                plan = ot.emd(
                    np.ascontiguousarray(a_npsd[i, :]),
                    np.ascontiguousarray(b_npsd[i, :]),
                    cost,
                    check_marginals=False
                )
                total_cost += np.sum(plan * cost)

            return 1-np.log1p(np.abs(total_cost / a.shape[0]))

        elif method == "p-ot":
            # Wasserstein-1 earthmover's distance of probability distributions
            #   = integrate(tf.abs(antiderivative(pdf_1) - antiderivative(pdf_2)))
            # get cdfs of the distributions
            a_cdf = self._get_cdf_batch(a)
            b_cdf = self._get_cdf_batch(b)
            # get the absolute value of their difference
            diff = tf.cast(tf.abs(a_cdf - b_cdf), dtype=tf.complex64)
            # integrate their absolute difference
            abs_diff = tf.abs(self._integrate_batch(diff))
            # print(abs_diff[:5])
            return 1-np.log1p(tf.reduce_sum(abs_diff).numpy())

        else:
            raise ValueError(f"Unknown method: {method}")

### FuzzyFourierTensorTransformer

In [ ]:
class FuzzyFourierTensorTransformer:
    """
    TensorFlow-accelerated fuzzy tensor transformer.
    All operations are vectorized and GPU-compatible.
    """

    def __init__(self, sigma: float = 0.1, kernel_size: int = 8):
        self.fuzzifier = FourierFuzzifier(sigma, kernel_size)
        self.kernel_size = kernel_size

    @tf.function
    def fuzzify(self, A: tf.Tensor) -> tf.Tensor:
        """
        Vectorized fuzzification.
        A: (d)
        Returns: shape (d, kernel_size)
        """
        if len(tf.shape(A)) != 1:
            raise ValueError(f"Input tensor must have shape (d), received tensor of shape {tf.shape(A)}")

        # Reshape to (batch_size, d, kernel_size)
        return self.fuzzifier._get_gaussian_at_mu_batch(A)


    @tf.function
    def intersection(self, A: tf.Tensor, B: tf.Tensor, normalize: bool = True) -> tf.Tensor:
        """
        Vectorized fuzzy intersection.
        A, B: shape (d,kernel_size)
        """
        if not len(A.shape) == 2:
          raise ValueError(f"A must be rank 2 tensors. Expected A.shape == 2, but got A.shape == {A.shape}")
        if not len(B.shape) == 2:
          raise ValueError(f"B must be rank 2 tensors. Expected A.shape == 2, but got A.shape == {B.shape}")

        result = self.fuzzifier._convolve_batch(A, B)

        if normalize:
            result = self.fuzzifier._normalize_batch(result)

        return result


    @tf.function
    def iterated_intersection(self, vects: tf.Tensor) -> tf.Tensor:
        """
        Efficiently compute intersection over multiple tensors.
        vects: shape (n_vects, d,kernel_size)
        """

        # if there's only 1 tensor to get the intersection
        if tf.shape(vects)[0] == 1:
            return vects[0]

        result = vects[0]
        for v in vects:
          # only include vect in intersection if it's the correct shape
          if len(v.shape) == 3:
              result = self.intersection(result, v, normalize=False)

        # Normalize the final result
        return self.fuzzifier._normalize_batch(result)

    @tf.function
    def union(self, A: tf.Tensor, B: tf.Tensor, normalize: bool = True) -> tf.Tensor:
        """
        Vectorized fuzzy union: A + B - A*B
        A, B: shape (d,kernel_size)
        """
        convolved = self.intersection(A, B, normalize=False)
        result = A + B - convolved

        if normalize:
            shape = tf.shape(result)
            if len(result.shape) == 3:
                result = self.fuzzifier._normalize_batch(result)
            else:
                result_flat = tf.reshape(result, [-1, self.kernel_size])
                result_flat = self.fuzzifier._normalize_batch(result_flat)
                result = tf.reshape(result_flat, shape)

        return result

    @tf.function
    def iterated_union(self, vects: tf.Tensor) -> tf.Tensor:
        """
        Efficiently compute union over multiple tensors.
        vects: shape (n_vects, d,kernel_size)
        """

        # if there's only 1 tensor to get the union
        if tf.shape(vects)[0] == 1:
            return vects[0]

        result = vects[0]
        for v in vects:
          # only include vect in union if it's the correct shape
          if len(v.shape) == 3:
              result = self.union(result, v, normalize=False)

        # Normalize the final result
        return self.fuzzifier._normalize_batch(result)

    def similarity(self, A: tf.Tensor, B: tf.Tensor, method:str = "p-ot") -> float:
        """
        Vectorized similarity computation.
        A, B: shape (d,kernel_size)
        Returns: scalar similarity between the normalized power spectral densities of A, B
        """
        if not len(A.shape) == 2:
          raise ValueError(f"A must be rank 2 tensors. Expected A.shape == 2, but got A.shape == {A.shape}")
        if not len(B.shape) == 2:
          raise ValueError(f"B must be rank 2 tensors. Expected A.shape == 2, but got A.shape == {B.shape}")

        if method == "cos":
            a_npsd = self.fuzzifier.get_npsd_batch(A)
            b_npsd = self.fuzzifier.get_npsd_batch(B)
            # numerator = aggregated hadamard product of a_npsd and b_npsd
            numerator = tf.reduce_sum(a_npsd * b_npsd)
            denominator_a = tf.sqrt(tf.reduce_sum(a_npsd * a_npsd))
            denominator_b = tf.sqrt(tf.reduce_sum(b_npsd * b_npsd))
            # similarity = correllation coefficient between the two npsd's
            similarity = numerator / (denominator_a * denominator_b + 1e-10)
            return similarity.numpy()

        else:
            return self.fuzzifier.similarity_batch(A, B, method)

## SpacyDependencyComposer

For assigning root of glosses' composition trees:
*   if the synset is for a verb, the first verb token is (almost) always the synset referent
*   if the synset is for an adverb, the first token after the word "with"/"without" is (almost) always the synset referent
*   if the synset is for an adjective, the first verb or adjective token is (almost) always the synset referent
*   if the synset is for a noun, the first noun token is (almost) always the synset referent


In [ ]:
strategies = [
      None,
      "mean",
      "intersection+mean",
      "intersection+union",
      "intersection+intersection",
      "selective_intersection+mean",
      "selective_intersection+union",
      "selective_intersection+intersection+mean",
  ]

In [ ]:
class SpacyDependencyComposer:
  def __init__(self,
      strategy:str=None,
      spacy_model=None,
      lemma_vectorizer=None,
      fuzzifier=None
    ):
    if strategy not in strategies:
      raise ValueError(f"Unknown strategy: {strategy}")
    self.strategy = strategy
    self.nlp = spacy.load("en_core_web_sm") if spacy_model is None else spacy_model
    self.lemma_vectorizer = LemmaVectorizer() if lemma_vectorizer is None else lemma_vectorizer
    self.fuzzifier = FuzzyFourierTensorTransformer() if fuzzifier is None else fuzzifier

  def _compose_tok_embedding(self, token) -> tf.Tensor:
    # lemma_vectorizer returns a numpy array, so convert it to tf.Tensor for fuzzification
    current_tok_tens = self.lemma_vectorizer(token.lemma_.lower())
    current_tok_tens = tf.convert_to_tensor(current_tok_tens, dtype=tf.float32)
    current_tok_tens = self.fuzzifier.fuzzify(current_tok_tens)

    if token.children:
      # get all the childrens'embeddings
      child_embeddings = [
          self._compose_tok_embedding(c)
          for c in token.children
      ]

      if not child_embeddings:
        # If no valid children, return the current token's embedding
        return current_tok_tens

      # compose child embeddings based on strategy
      match self.strategy:

        case "mean":
          return tf.reduce_mean(tf.stack(child_embeddings), axis=0)

        case "intersection+mean":
          child_embeddings_intersected = [
              self.fuzzifier.intersection(current_tok_tens, c)
              for c in child_embeddings
          ]
          return tf.reduce_mean(tf.stack(child_embeddings_intersected), axis=0)

        case "intersection+union":
          child_embeddings_intersected = [
              self.fuzzifier.intersection(current_tok_tens, c)
              for c in child_embeddings
          ]
          return self.fuzzifier.iterated_union(child_embeddings_intersected)

        case "intersection+intersection":
          child_embeddings_intersected = [
              self.fuzzifier.intersection(current_tok_tens, c)
              for c in child_embeddings
          ]
          return self.fuzzifier.iterated_intersection(child_embeddings_intersected)

        case "selective_intersection+mean":
          # if the current token is a modifier of some kind, intersect it with all its children
          if token.pos_ in {"VERB", "ADJ", "ADV"}:
            child_embeddings_intersected = [
                self.fuzzifier.intersection(current_tok_tens, c)
                for c in child_embeddings
            ]
            return tf.reduce_mean(tf.stack(child_embeddings_intersected), axis=0)
          # otherwise, just return the mean of its children
          return tf.reduce_mean(tf.stack(child_embeddings), axis=0)

        case "selective_intersection+union":
          # if the current token is a modifier of some kind, intersect it with all its children
          if token.pos_ in {"VERB", "ADJ", "ADV"}:
            child_embeddings_intersected = [
                self.fuzzifier.intersection(current_tok_tens, c)
                for c in child_embeddings
            ]
            # return the union of these intersected children
            return self.fuzzifier.iterated_union(child_embeddings_intersected)
          # otherwise, just return the union of its children
          return self.fuzzifier.iterated_union(tf.stack(child_embeddings))

        case "selective_intersection+intersection+mean":
          # if the current token is a modifier of some kind, intersect it with all its children
          if token.pos_ in {"VERB", "ADJ", "ADV"}:
            child_embeddings_intersected = [
                self.fuzzifier.intersection(current_tok_tens, c)
                for c in child_embeddings
            ]
            # then intersect those intersections
            return self.fuzzifier.iterated_intersection(child_embeddings_intersected)
          # otherwise, just return the mean of its children
          return tf.reduce_mean(tf.stack(child_embeddings), axis=0)

        case _:
          raise ValueError(f"Unknown strategy: {self.strategy}")

    # if the token is a leaf
    else:
      return current_tok_tens

  def __call__(self,
      text: str,
  ) -> tf.Tensor:
      # baseline
      if self.strategy is None:
          return self.fuzzifier.fuzzify(self.lemma_vectorizer(text))

      doc = self.nlp(text)
      root_embeddings = []
      for sent in doc.sents:
          root_emb = self._compose_tok_embedding(sent.root)
          if root_emb is not None:
            root_embeddings.append(root_emb)

      if not root_embeddings:
          return None

      # get the average
      return tf.reduce_mean(tf.stack(root_embeddings), axis=0)

# Vectorize and fuzzify wn_df entries

## Config

In [ ]:
def vectorize(strings: List[str]) -> tf.Tensor:
  vects = embedding_model.encode(strings)
  normalized_vects = (vects - np.min(vects))/(np.max(vects) - np.min(vects))
  return tf.convert_to_tensor(normalized_vects)

In [ ]:
spacy_model = spacy.load("en_core_web_sm")
lemma_vectorizer = LemmaVectorizer()
fuzzifier = FuzzyFourierTensorTransformer()

In [ ]:
composers = {
    s: SpacyDependencyComposer(
        strategy=s,
        spacy_model=spacy_model,
        lemma_vectorizer=lemma_vectorizer,
        fuzzifier=fuzzifier
    )
    for s in strategies
}

gloss_vect_fuzzy_cols = {
    s: f"gloss_fuzzy_{s}"
    for s in strategies
}

## Vectorize, fuzzify

In [ ]:
# get mean embedding for synset lemmas
wn_df['synset_vect'] = wn_df['synset'].apply(lambda x: vectorize(x.split(".")[0].replace("_", " ")))
wn_df['gloss_vect'] = wn_df['gloss'].apply(vectorize)
wn_df['lemmas_vect'] = wn_df['lemmas'].apply(vectorize)

In [ ]:
wn_df.head()

In [ ]:
# get fuzzified synset embedding
wn_df['synset_vect_fuzzy'] = wn_df['synset_vect'].apply(lambda x: fuzzifier.fuzzify(x))
# get compositional embeddings for glosses using different strategies
for s, composer in composers.items():
  print(f"Getting gloss embedding with {s} approach...")
  wn_df[gloss_vect_fuzzy_cols[s]] = wn_df["cleaned_gloss"].apply(composer)

In [ ]:
print("Columns:")
for c in wn_df.columns:
  print(f"\t{c}")

# Test fuzzy embedding operators

In [ ]:
# pick a random row in wn_df
row_idx = random.choice(wn_df.index)
# get the fourier tensors for the first lemma's fuzzy vect
fuzzy_vects = wn_df.loc[row_idx, 'synset_vect_fuzzy']
intersected_components = fuzzifier.fuzzifier.intersection(fuzzy_vects[0], fuzzy_vects[1])

In [ ]:
print(f"Area under first component of synset {row_idx}: {fuzzifier.fuzzifier._integrate(fuzzy_vects[0])}")
print(f"Area under second component of synset {row_idx}: {fuzzifier.fuzzifier._integrate(fuzzy_vects[1])}")
print(f"Area under intersection of first 2 components of synset {row_idx}: {fuzzifier.fuzzifier._integrate(intersected_components)}")

# Evaluate Inner Product (Similarity) Preservation

## Config

In [ ]:
fuzzifier = FuzzyFourierTensorTransformer()
# npsd-ot seems to do best for the fuzzy intersection+mean composition model
sim_metric = "cos"
fuzzifier.fuzzifier.kernel_size

In [ ]:
sim = tf.keras.losses.CosineSimilarity(
    axis=-1,
    reduction='sum_over_batch_size',
    name='cosine_similarity'
)
def cosine_similarity(a: tf.Tensor, b: tf.Tensor) -> tf.Tensor:
  return -1*sim(a, b).numpy()

In [ ]:
random_pairing_indices = [
    tuple(random.sample(range(len(wn_df)), 2))
    for i in range(sample_size)
]

sim_eval_dict = {
    "syn_gloss_cos_sim": [],
    "is_related": [],
}

for s in strategies:
  # for comparing inner product of fuzzy synset similarities to related glosses
  sim_eval_dict[f"syn_{gloss_vect_fuzzy_cols[s]}"] = []

## Fuzzy space similarities

In [ ]:
# extend classes
sim_eval_dict["is_related"] = [1, 0] * len(random_pairing_indices)
# syn-gloss cosine similarities
for a, b in random_pairing_indices:
  sim_eval_dict["syn_gloss_cos_sim"].extend([
      cosine_similarity(
          wn_df.loc[a, 'synset_vect'],
          wn_df.loc[a, 'gloss_vect']
      ),
      cosine_similarity(
          wn_df.loc[a, 'synset_vect'],
          wn_df.loc[b, 'gloss_vect']
      )
  ])

In [ ]:
# syn-gloss custom fuzzy similarities
import numpy as np # Ensure numpy is imported for np.nan

for s in strategies:
  print(f"Evaluating similarities for gloss embeddings with {s} composition strategy...")

  for a, b in random_pairing_indices:
    synset_vect_fuzzy = wn_df.loc[a, 'synset_vect_fuzzy']
    fuzzy_col = gloss_vect_fuzzy_cols[s]

    gloss_a_fuzzy = wn_df.loc[a, fuzzy_col]
    gloss_b_fuzzy = wn_df.loc[b, fuzzy_col]

    sim_a = np.nan # Default to NaN
    sim_b = np.nan # Default to NaN

    # Only calculate similarity if the gloss embedding is not None
    if gloss_a_fuzzy is not None:
        sim_a = fuzzifier.similarity(
            synset_vect_fuzzy,
            gloss_a_fuzzy,
            sim_metric
        )

    # Only calculate similarity if the gloss embedding is not None
    if gloss_b_fuzzy is not None:
        sim_b = fuzzifier.similarity(
            synset_vect_fuzzy,
            gloss_b_fuzzy,
            sim_metric
        )

    sim_eval_dict[f"syn_{fuzzy_col}"].extend([sim_a, sim_b])

In [ ]:
sim_eval_df = pd.DataFrame.from_dict(sim_eval_dict)
# normalize similarities
for col in sim_eval_df.columns:
  sim_eval_df[col] = (sim_eval_df[col] - sim_eval_df[col].min()) / (sim_eval_df[col].max() - sim_eval_df[col].min())
sim_eval_df.head()

In [ ]:
sim_eval_df.describe()

In [ ]:
for c in sim_eval_df.columns:
  print(f"Number of np.nans: {np.sum(sim_eval_df[c].isna())}")

## Visualize embedding similarities

In [ ]:
cmap = plt.get_cmap("viridis")
colors = cmap(np.linspace(0, 1, len(sim_eval_df.columns)))
for i, col in enumerate(sim_eval_df.columns):
  if col in ["syn_gloss_cos_sim", "is_related"]:
    continue
  plt.scatter(
      x=sim_eval_df["syn_gloss_cos_sim"],
      y=sim_eval_df[col],
      color=colors[i],
      label=col
  )

plt.xlabel("synset-gloss cosine imilarity (sentence transformer)")
plt.ylabel("fuzzy synset-gloss similarity (compositional)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.yscale("log")
plt.title("synset-gloss Fuzzy Similarity vs. synset-gloss Fuzzy Compositional Similarity")
plt.show()

In [ ]:
sim_eval_df.columns

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

plt.figure(figsize=(16, 8))

plt.title(f"Confusion Matrices for different embedding composition methods using {sim_metric} similarity\n")
for i, col in enumerate(sim_eval_df.columns):
    if col in ["is_related"]:
      continue

    # Calculate confusion matrix
    cm = confusion_matrix(
        sim_eval_df['is_related'],
         (sim_eval_df[col] >= sim_eval_df[col].mean()).astype(int)
    )

    # Plot confusion matrix
    plt.subplot(4, 3, i+1)
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='viridis',
        xticklabels=['Unrelated (0)', 'Related (1)'],
        yticklabels=['Unrelated (0)', 'Related (1)']
    )
    plt.title(f'Confusion Matrix for {col.replace("syn_gloss_", "").replace("_", " ")}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')

plt.tight_layout()
plt.show()

# Stat analysis

## Correlation coefficients

In [ ]:
plt.figure(figsize=(10, 10))
ax = sns.heatmap(sim_eval_df.corr(), annot=True)
ax.tick_params(axis='x', labelrotation=30)
plt.title(f"Correlation coefficients between compositional models' similarity estimates for the {sim_metric} metric\n")
plt.show()

## Comparing composition strategies' differentiation capability between related and unrelated glosses

In [ ]:
sep_eval_dict = {
    "mean_cos_similarity": [],
    "cos_separability": []
}
for s in strategies:
  sep_eval_dict[f"separability_{s}"] = []

In [ ]:
for i in range(0, len(sim_eval_df), 2):
  # cosine similarity
  sep_eval_dict["mean_cos_similarity"].append(sim_eval_df.loc[i, "syn_gloss_cos_sim"]/2 + sim_eval_df.loc[i+1, "syn_gloss_cos_sim"]/2)
  sep_eval_dict["cos_separability"].append(sim_eval_df.loc[i, "syn_gloss_cos_sim"] - sim_eval_df.loc[i+1, "syn_gloss_cos_sim"])
  for s in strategies:
    strat = f"syn_{gloss_vect_fuzzy_cols[s]}"
    sep_eval_dict[f"separability_{s}"] = sim_eval_df.loc[i, strat] - sim_eval_df.loc[i+1, strat]

sep_eval_df = pd.DataFrame.from_dict(sep_eval_dict)

## Visuals

In [ ]:
for c in sep_eval_df.columns:
  if c in ["mean_cos_similarity"]:
    continue
  plt.scatter(
      x=sep_eval_df["mean_cos_similarity"],
      y=sep_eval_df[c],
      label=c
  )
plt.xlabel("Mean cosine similarity")
plt.ylabel("Similarity difference")
# plt.xscale("log")
plt.yscale("log")
plt.legend(
    [c for c in sep_eval_df.columns if c not in ["mean_cos_similarity"]],
    bbox_to_anchor=(1.05, 1),
    loc='upper left'
)
plt.title(f"Similarity difference of different embedding composition methods for the {sim_metric} metric\n")
plt.show()

del sep_eval_df["mean_cos_similarity"]

In [ ]:
for col in sep_eval_df.columns:
  plt.hist(
      sep_eval_df[col],
      bins=100,
      alpha=0.8,
      # range=(-.01, .01)
  )

plt.xlabel("Similarity difference")
plt.ylabel("Frequency")
# plt.xscale("log")
plt.yscale("log")
plt.legend(
    sep_eval_df.columns,
    bbox_to_anchor=(1.05, 1),
    loc='upper left'
)
plt.title(f"Similarity difference of different embedding composition methods with {sim_metric} similarity")
plt.show()

## Significance tests for separability

In [ ]:
print("p-value for separability of...")
for col in sep_eval_df.columns:
  # get probability mean(eval_df["separability_fuzzy"]) > 0
  p = stats.ttest_1samp(
      sep_eval_df[col],
      0,
      alternative="greater"
  ).pvalue
  print(f"\t{col} > 0:{'\t'*(10 - len(col)//7)}{p}")